<a href="https://colab.research.google.com/github/PatrickMarinich/Mario-Kart-Stats-Project/blob/main/Kartnite_Stats_v2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Kartnite Stats** v2.2

*Written by: Patrick Marinich*

**Goal:** This scripts goal is to take user inputted *Mario Kart Wii* races and other stats and save them to a database to be used for calculations and analysis when user requested. 

**Current Functionalities**


*   Take in a user inputted race, with the track, players, and scores, and adds them to the database of all scores
*   Keep track of how many times each player playes on a singular track
* Get the average score per track of all of the players
* Determine which player is the best on any specific track
* A leaderboard to display the ordering of player seeding (power rankings)
* The ability to search for all the stats of a specifc track, including total points, player averages, and the individual track leaderboards
* The ability to search of all of the stats of a individual player, including total points, track averages, Track MVPs, and total power ranking points (Kart Score)
* The ability to correct mistakes in entering race points, by having a user setting to manually edit the points of a specific player
* Ability to view the database containing all of the data





**Important:** Make sure to run all cells before running the main cell, the main cell is the last one. If all function cells are not run then the main cell will break and throw errors



**Patch Notes**

v2.2 (Jan 7 2022)
- Re-design of player stats output
- Re-orginization of code execution order for determining when the database is updated compared to finding track MVPs
-Points Per Race Leaderbaord
-A Feature to view all track MVPs at once
-Re-design of user input options
-Track Nickname functionality for user inputs (currently 77 working track nicknames)


v2.1 (Jan 4 2022)
- Optimization
- Changes to 'Kart Score' 
- MVP Leaderboards Per Track

v2.0 (Jan 3 2022)
 - Adds Shock Dodges 
 - Adds GP Wins
 - Adds Total Race Count
 - Leaderboards

v1.1 (Jan 2nd 2022)
 - Moved the Main Method into a private method, this reduced the need for scrolling

v1.0 (December 27 2021 - December 31st 2021)
 - Importing and Saving Race Data
 - Editing singular race scores incase of mistakes
 - Viewing Track Records + Best Player on the Track
 - Viewing Player Records


In [5]:
def inputRace(currentData,RaceCount,dfKartScore, Track, Racers, Scores,TrackIndex):  
  #checks to see if the track exists
  if Track in set(currentData["Tracks x Players"]):
    print('Valid', Track)
    #seperates the players into an array, make sure that all players also exist

    racersArray = Racers.split()
    # checks that all players are valid
    validCount = 0
    for racer in racersArray:
      if(racer in set(currentData.columns.values.tolist())):
          print('Valid', racer)
          validCount = validCount + 1
      else: 
          validCount = validCount - 1
          print('Invalid', racer)

  
    if validCount == len(racersArray):
      print("....Everything is valid, Updating the scores....")

      ##if all players are valid update the scores

      #gets scores into an array
      newScores = Scores.split()
      #changes to integers
      for i in range(len(newScores)):
        newScores[i] = int(newScores[i])
      #Adds new Score to the current score for each racer, uses the track index to find the track
      for i in range(len(racersArray)):
        
        #only gives the points if there is a singular MVP
        if(len(getTrackOwner(currentData,RaceCount, Track, TrackIndex).split()) == 1):
        #gives the track owner 2.5 points per player
          dfKartScore.at[0, getTrackOwner(currentData,RaceCount, Track, TrackIndex)] = int(dfKartScore.at[0, getTrackOwner(currentData,RaceCount, Track, TrackIndex)]) + 2


        #increments the score
        currentData.at[TrackIndex[Track],racersArray[i]] =  int(currentData.at[TrackIndex[Track],racersArray[i]]) + newScores[i]
        #increments the count
        RaceCount.at[TrackIndex[Track],racersArray[i]] =  int(RaceCount.at[TrackIndex[Track],racersArray[i]]) + 1
       
        
    else: 
      print('There was an error entering the racers, please try again')
  else:
    print('There was an error entering the track name, please try again:', Track, "was invalid")





In [6]:
def editAScore(dfScores, Track, Racer, Score,TrackIndex):
 #makes sure track is valid
  if Track in set(dfScores["Tracks x Players"]):
    print('Valid', Track)
    # makes sure racer is valid
    if Racer in set(dfScores.columns.values.tolist()):
      print('Valid', Racer)
      #changes a specific score
      dfScores.at[TrackIndex[Track],Racer] =  int(dfScores.at[TrackIndex[Track],Racer]) + int(Score)
      print(Racer, '\'s Score was changed by ', Score, ' on ', Track)
    else:  
      print('There was an error entering the racer, please try again')
  else:
     print('There was an error entering the track name, please try again:', Track, "was invalid")



In [7]:
def getPlayerAverage(dfScores, dfRaceCount, Player, Track, TrackIndex):
  #eliminates the first row, to allow for players only
  dfNoTracks = dfScores[dfScores.columns.difference(["Tracks x Players"])]
  
  if Player in set(dfNoTracks.columns.values.tolist()):
    #checks for divide by 0

    RaceCount = int(dfRaceCount.at[TrackIndex[Track],Player])
    Score =  int(dfScores.at[TrackIndex[Track],Player])
   
    if(Score == 0 or RaceCount == 0):
      averageScore = 0
    else: 
      averageScore = Score/RaceCount
  
    

  else:
    averageScore = "ERROR"
    print("There was an internal with Averaging Have Pat re-read code")
  

  return averageScore



In [8]:
def getTrackOwner(dfScores,dfRaceCount, Track, TrackIndex):
  #eliminates the first row, to allow for players only
  dfNoTracks = dfScores[dfScores.columns.difference(["Tracks x Players"])]


  #goes through all players and then find the max score

  currentMaxScore = 0
  currentPlayer = ""
  TrackTotalPoints = 0
  for player in set(dfNoTracks.columns.values.tolist()):
    TrackTotalPoints = TrackTotalPoints + int(dfScores.at[TrackIndex[Track], player])

  for player in set(dfNoTracks.columns.values.tolist()):
    #way to calculate track owner, total points + average
    
    playerScore = ((int(dfScores.at[TrackIndex[Track],player])/TrackTotalPoints)*100) + getPlayerAverage(dfScores,dfRaceCount,player,Track,TrackIndex)
  
    if playerScore> currentMaxScore:
      currentPlayer = player
      currentMaxScore = playerScore
    elif playerScore == currentMaxScore:
      currentPlayer = currentPlayer + ", " + player
    else:
      continue


  return currentPlayer



In [9]:
def GetTrackData(dfScores,dfRaceCount,Track,TrackIndex):
 
  #eliminates the first row, to allow for players only
  dfNoTracks = dfScores[dfScores.columns.difference(["Tracks x Players"])]
 
  #make sure that the track exists in the data
  if Track in set(dfScores["Tracks x Players"]):
    print("Stats For: ", Track)
    #now display the data for the track plus any interesting stats
    print("\n", "Total Scores")
    print(dfScores.loc[[TrackIndex[Track]]])

    races = 0
    for player in set(dfNoTracks.columns.values.tolist()):
      races = races + int(dfRaceCount.at[TrackIndex[Track], player])
    print('\nTimes Played:',races )

    #prints player averages. 
    print('\nPlayer Averages:')
    for racer in set(dfNoTracks.columns.values.tolist()):
      print(racer,": ", getPlayerAverage(dfScores,dfRaceCount,racer,Track,TrackIndex))
    print('\n')
    print(getTrackOwner(dfScores, dfRaceCount, Track, TrackIndex), "is the MVP for this track")
    
    print('\nMVP Leaderboard')
    #determine the score for each player within the data
    dfNoTracks = dfScores[dfScores.columns.difference(["Tracks x Players"])]
    #leaderboard object
    dfLeaderboard = pd.DataFrame({'Player': [], 'MVP Points':[]})
    counter = 0
   
    #gets the ownership scores of each player for the track
    TrackTotalPoints = 0
    for player in set(dfNoTracks.columns.values.tolist()):
      TrackTotalPoints = TrackTotalPoints + int(dfScores.at[TrackIndex[Track], player])
    for player in set(dfNoTracks.columns.values.tolist()):
      ownershipScore = ((int(dfScores.at[TrackIndex[Track],player])/TrackTotalPoints)*100) + getPlayerAverage(dfScores,dfRaceCount,player,Track,TrackIndex)


      #puts the score into the datafram
      dfLeaderboard.loc[counter] = [player, ownershipScore]
      counter = counter + 1
      playerScore = 0

    #sorts and prints the leaderboards
    dfLeaderboard = dfLeaderboard.sort_values(['MVP Points', 'Player'],  ascending=[0, 1])
    print(dfLeaderboard)

  else:
    print('Track Selection Was Invalid')


In [10]:
def getPlayerStats(dfScores,dfRaceCount, dfWins,dfShock, dfKartScore, Player, TrackIndex):
  #prints the total Scores
  dfPlayerScores = dfScores[["Tracks x Players", Player]]
  print(Player, "\'s Total Scores")
  print("\n")
  print(dfPlayerScores)

  #prints Averages
  print("\n")
  print(Player, "\'s Track Averages")
  print("\n")
  for track in TrackIndex:
    print(track, "- ", getPlayerAverage(dfScores,dfRaceCount,Player,track,TrackIndex))



  print("\n")
  print("---------------")
  #gets all of the tracks that the player owns
  Owns = ''
  for track in TrackIndex:
    if getTrackOwner(dfScores,dfRaceCount,track,TrackIndex) == Player:
      
      if Owns == '':
        Owns = track
      else:
        Owns = Owns + "\n" + track

  if(Owns == ''):
    print("Track MVP on: None")
  else:
    print("Tracks MVP on:\n", Owns)

  print("---------------")

  #prints total wins
  totalRaces = 0
  for Track in TrackIndex:
    totalRaces = totalRaces + int(dfRaceCount.at[TrackIndex[Track],Player])
  print("Total Races Played:", totalRaces)

  #Gp Wins
  print("Grand Prix Wins:", dfWins.at[0,Player])
  #dodges
  print("Shock Dodges:", dfShock.at[0,Player])

  ####KART SCORE####
  dfNoTracks = dfScores[dfScores.columns.difference(["Tracks x Players"])]
  KartScore = 0
  #gp wins (+100 per win)
  WinsScore = int(dfWins.at[0,Player])*100
    

  #owner score (+2 per player per race)
  OwnedScore = int(dfKartScore.at[0, Player])
     
      

  #Dodges(+2 per dodge)
  DodgeScore = int(dfShock.at[0,Player])*2
    
  #total points(+1 per point)
  PointsScore = 0
  for track in TrackIndex:
   PointsScore = PointsScore + int(dfScores.at[TrackIndex[track], Player])

  #adds in track averages (+4 per) avg point
  avgScore = 0
  for track in TrackIndex:
    avgScore = avgScore + (int(getPlayerAverage(dfScores,dfRaceCount,Player,track,TrackIndex))*4)
  
  KartScore = WinsScore + OwnedScore + DodgeScore + avgScore  + PointsScore
  print('---------------')
  print("Kart Score:", KartScore)
  print('---------------')
  print("Kart Score Breakdown:")
  print('GP Win Points:', WinsScore)
  print('Points From Being Track MVP:', OwnedScore)
  print('Race Points:', PointsScore)
  print('Track Avg Points:', avgScore)
  print('Shock Dodge Points:', DodgeScore)




In [11]:
#allows for GP winners to be counted
def enterWinner(dfWins, Player):

  if Player in set(dfWins.columns.values.tolist()):
    
    dfWins.at[0, Player] = int(dfWins.at[0,Player]) + 1
    print(Player, 'has', dfWins.at[0,Player], 'wins')


  else: 
    print(Player, "does not exist within the Database, Please try again")



In [12]:
#allowes for Shock dodges
def enterDodges(dfShock, Player, Count):
  if Player in set(dfShock.columns.values.tolist()):
    
    dfShock.at[0, Player] = int(dfShock.at[0,Player]) + int(Count)
    print(Player, 'has', dfShock.at[0,Player], 'shock dodges')


  else: 
    print(Player, "does not exist within the Database, Please try again")


In [13]:
def getSeedings(dfScores,dfRaceCount,dfWins,dfShock,dfKartScore,TrackIndex):
    
    #determine the score for each player within the data
    dfNoTracks = dfScores[dfScores.columns.difference(["Tracks x Players"])]
    #leaderboard object
    dfLeaderboard = pd.DataFrame({'Player': [], 'Kart Score':[]})
    counter = 0
   
    #sets up the scores for each player
    for player in set(dfNoTracks.columns.values.tolist()):
      playerScore = 0
      #gp wins (+100 per win)
      playerScore = int(dfWins.at[0,player])*100
    
      #enters the owned track score (+2 per player per race)
      playerScore = playerScore + int(dfKartScore.at[0, player])


      #Dodges(+2 per dodge)
      playerScore = int(dfShock.at[0,player])*2 + playerScore
    
      #total points(+1 per point)
      points = 0
      for track in TrackIndex:
        points = points + int(dfScores.at[TrackIndex[track], player])
      playerScore = playerScore + points

      #adds in track averages (+4 per) avg point
      avgpoints = 0
      for track in TrackIndex:
        avgpoints = avgpoints + (int(getPlayerAverage(dfScores,dfRaceCount,player,track,TrackIndex))*4)
      playerScore = playerScore + avgpoints

      #puts the score into the datafram
      dfLeaderboard.loc[counter] = [player, playerScore]
      counter = counter + 1
      playerScore = 0

    #sorts and prints the leaderboards
    dfLeaderboard = dfLeaderboard.sort_values(['Kart Score', 'Player'],  ascending=[0, 1])
    print(dfLeaderboard)





In [14]:
def getRaceCountLeaderbaords(dfRaceCount,TrackIndex):
  
    #determine the score for each player within the data
    dfNoTracks = dfRaceCount[dfRaceCount.columns.difference(["Tracks x Players"])]
    #leaderboard object
    dfLeaderboard = pd.DataFrame({'Player': [], 'Races Played':[]})
    counter = 0
   
    racecount = 0
    #sets up the scores for each player
    for player in set(dfNoTracks.columns.values.tolist()):
      
      for track in TrackIndex:
        racecount = racecount + int(dfRaceCount.at[TrackIndex[track], player])


      #adds the racecount to the leaderboard
      dfLeaderboard.loc[counter] = [player, racecount]
      counter = counter + 1
      racecount = 0
   
    #sorts and prints the leaderboards
    dfLeaderboard = dfLeaderboard.sort_values(['Races Played', 'Player'],  ascending=[0, 1])
    print(dfLeaderboard)


In [15]:
def getGPWinsLeaderboard(dfWins):
    #leaderboard object
    dfLeaderboard = pd.DataFrame({'Player': [], 'Grand Prix Wins':[]})
    counter = 0
    #for each player in wins
    for player in set(dfWins.columns.values.tolist()):
      #adds the wins to the leaderboard
      dfLeaderboard.loc[counter] = [player, dfWins.at[0, player]]
      counter = counter + 1
      racecount = 0
   
    #sorts and prints the leaderboards
    dfLeaderboard = dfLeaderboard.sort_values(['Grand Prix Wins', 'Player'],  ascending=[0, 1])
    print(dfLeaderboard)


In [16]:
def getShockDodges(dfShock):
  #leaderboard object
    dfLeaderboard = pd.DataFrame({'Player': [], 'Shock Dodges':[]})
    counter = 0
    #for each player in dodges
    for player in set(dfShock.columns.values.tolist()):
      #adds the dodges to the leaderboard
      dfLeaderboard.loc[counter] = [player, dfShock.at[0, player]]
      counter = counter + 1
      racecount = 0
   
    #sorts and prints the leaderboards
    dfLeaderboard = dfLeaderboard.sort_values(['Shock Dodges', 'Player'],  ascending=[0, 1])
    print(dfLeaderboard)



In [17]:
def getPointsPerRace(dfScores,dfRaceCount, TrackIndex):
  #creates a leaderboard object
  dfLeaderboard = pd.DataFrame({'Player': [], 'Points Per Race': []})
  counter = 0


   #dataframe without the track names
  dfNoTracks = dfScores[dfScores.columns.difference(["Tracks x Players"])]

  #iterates through each player
  for player in set(dfNoTracks.columns.values.tolist()):

    #gets the total points of each player
    totalPoints = 0
    for track in TrackIndex:
      totalPoints = totalPoints + int(dfScores.at[TrackIndex[track],player])
    #gets total racecount
    totalRaces = 0
    for track in TrackIndex:
      totalRaces = totalRaces + int(dfRaceCount.at[TrackIndex[track],player])

    #gets the PPR and then sets it to the approperate place
    
    if (totalRaces != 0):
      PPR = float(totalPoints) / float(totalRaces)
      dfLeaderboard.loc[counter] = [player, PPR]
      counter = counter + 1
    else:
      dfLeaderboard.loc[counter] = [player,0]
      counter = counter + 1
     
    

   #sorts and prints the leaderboard
  dfLeaderboard = dfLeaderboard.sort_values(['Points Per Race', 'Player'], ascending=[0,1]) 
  print(dfLeaderboard)
    




In [18]:
def getAllTrackOwners(dfScores,dfRaceCount,TrackIndex):
  #the list that will get all of the track mvps
  dfList = pd.DataFrame({'Track': [], 'Current MVP': []})

  counter = 0
  for track in TrackIndex:
    mvp = getTrackOwner(dfScores,dfRaceCount, track, TrackIndex)
    dfList.loc[counter] = [track,mvp]
    counter = counter + 1


  print(dfList)

In [33]:
#This is the main bulk of the code, it culminates all of the previous functions 
#into the user inputted choices, this is the method that gets user inputs and does all 
#of the database scraping and saving for this program to work.
def RunKartniteStats():
### Patrick Marinich December 2021

  #imports
  import pandas as pd
  from google.colab import auth
  auth.authenticate_user()
  import gspread
  from oauth2client.client import GoogleCredentials
  from gspread_dataframe import get_as_dataframe, set_with_dataframe

  # Puts the google sheet into a python object, we will be calling the sh object after a selection is made
  gc = gspread.authorize(GoogleCredentials.get_application_default())
  sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1G0z4XeBjG7Q_Zk_uPtV4v8BPkVT8GD9pmsuk7VIwFKc/edit#gid=0')

  #Track Dictionary, all track names and nicknames go here
  #track indexes are starting at 0 in the mushroom cup ending at 31 at N64 Bowsers castle, +32 for Lava Lake
  TrackIndex = {"Luigi Circuit":0,
  "Moo Moo Meadows":1,
  "Mushroom Gorge":2,
  "Toad's Factory":3,
  "Mario Circuit":4,
  "Coconut Mall":5,
  "DK Summit":6,
  "Wario's Gold Mine":7,
  "Daisy Circuit":8,
  "Koopa Cape":9,
  "Maple Treeway":10,
  "Grumble Volcano":11,
  "Dry Dry Ruins":12,
  "Moonview Highway":13,
  "Bowser's Castle":14,
  "Rainbow Road":15,
  "GCN Peach Beach":16,
  "DS Yoshi Falls":17,
  "SNES Ghost Valley 2":18,
  "N64 Mario Raceway":19,
  "N64 Sherbet Land":20,
  "GBA Shy Guy Beach":21,
  "DS Delfino Square":22,
  "GCN Waluigi Stadium":23,
  "DS Desert Hills":24,
  "GBA Bowser's Castle 3":25,
  "N64 DK's Jungle Parkway":26,
  "GCN Mario Circuit":27,
  "SNES Mario Circuit 3":28,
  "DS Peach Gardens":29,
  "GCN DK Mountain":30,
  "N64 Bowser's Castle":31,
  "Lava Lake":32 }

#for nicknames 
  NickNameIndex= {"Luigi":"Luigi Circuit",
   "moo moo": "Moo Moo Meadows",
   "Moo Moo": "Moo Moo Meadows",
   "Gorge":"Mushroom Gorge",
   "gorge":"Mushroom Gorge",
   "toads":"Toad's Factory",
   "Toads":"Toad's Factory",
   "toads factory": "Toad's Factory",
   "mario circuit": "Mario Circuit",
   "coconut mall":"Coconut Mall",
   "coconut":"Coconut Mall",
   "summit":"DK Summit",
   "Summit":"Dk Summit",
   "gold mine":"Wario's Gold Mine",
   "Gold Mine":"Wario's Gold Mine",
   "koopa": "Koopa Cape",
   "Koopa": "Koopa Cape",
   "maple":"Maple Treeway",
   "Maple":"Maple Treeway",
   "grumble":"Grumble Volcano",
   "Grumble":"Grumble Volcano",
   "Dry Dry":"Dry Dry Ruins",
   "dry dry":"Dry Dry Ruins",
   "Moonview":"Moonview Highway",
   "moonview":"Moonview Highway",
   "BC Wii":"Bowser's Castle",
   "bc wii":"Bowser's Castle",
   "BC wii":"Bowser's Castle",
   "rainbow road":"Rainbow Road",
   "rainbow":"Rainbow Road",
   "Rainbow":"Rainbow Road",
   "Peach Beach":"GCN Peach Beach",
   "peach beach":"GCN Peach Beach",
   "yoshi falls":"DS Yoshi Falls",
   "Yoshi Falls":"DS Yoshi Falls",
   "Ghost Valley":"SNES Ghost Valley 2",
   "ghost valley":"SNES Ghost Valley 2",
   "mario raceway":"N64 Mario Raceway",
   "raceway": "N64 Mario Raceway",
   "Sherbet Land":"N64 Sherbet Land",
   "sherbet land":"N64 Sherbet Land",
   "Shy Guy Beach":"GBA Shy Guy Beach",
   "shy guy beach":"GBA Shy Guy Beach",
   "Shy Guy":"GBA Shy Guy Beach",
   "shy guy":"GBA Shy Guy Beach",
   "Delfino Square":"DS Delfino Square",
   "delfino square":"DS Delfino Square",
   "Delfino":"DS Delfino Square",
   "delfino":"DS Delfino Square",
   "Waluigi Stadium":"GCN Waluigi Stadium",
   "waluigi stadium":"GCN Waluigi Stadium",
   "Waluigi":"GCN Waluigi Stadium", 
   "waluigi":"GCN Waluigi Stadium",
   "Desert Hills":"DS Desert Hills",
   "desert hills":"DS Desert Hills",
   "GBA 3":"GBA Bowser's Castle 3",
   "gba 3":"GBA Bowser's Castle 3",
   "BC3":"GBA Bowser's Castle 3",
   "bc3":"GBA Bowser's Castle 3",
   "Parkway":"N64 DK's Jungle Parkway",
   "parkway":"N64 DK's Jungle Parkway",
   "GCN mario circuit":"GCN Mario Circuit",
   "GCN mario":"GCN Mario Circuit",
   "gcn mario":"GCN Mario Circuit",
   "gcn Mario":"GCN Mario Circuit",
   "SNES 3":"SNES Mario Circuit 3",
   "snes 3":"SNES Mario Circuit 3",
   "Peach Gardens":"DS Peach Gardens",
   "peach gardens":"DS Peach Gardens",
   "DK Mountain": "GCN DK Mountain",
   "dk mountain": "GCN DK Mountain",
   "mountain": "GCN DK Mountain",
   "Mountain": "GCN DK Mountain",
   "BC64":"N64 Bowser's Castle",
   "bc64":"N64 Bowser's Castle",
   "N64BC":"N64 Bowser's Castle",
   "n64bc":"N64 Bowser's Castle"}

  #Below is the logic for asking a user what they would like to do
  ########


  #displaying inputs for the user
  print('Welcome to Kartnite Stats v2.2', "\nDeveloped by: Patrick Marinich\n")
  print('Below are the options, type the number of the option you would like')
  print('1. Input a Race \n2. Edit A Singular Score \n3. Enter A Grand Prix Winner \n4. Enter Shock Dodges \n5. View Leaderboards \n6. View All Databases \n7. View Track Records \n8. View Player Stats  \n9. View All Current Track MVPs')
 
  selection = input('What Would You Like To Do: ')
  
  #selection logic
  
  #depending on the user selection it will ask for different inputs, and load the nessassary databases needed for that operation

  #input a race
  if selection == '1':
    print('Warining: track and racer names must be formatted correctally \nTrack names must match their full name or a registered nickname \nRacer names must have their first letter capital')
    Track = input("Name of Track: ")
    if Track in NickNameIndex:
      Track = NickNameIndex[Track]
    Racers = input("Names of Racers: ")
    Scores = input("Scores:")
    #loading statment
    print("\nYour selection of", selection, "is loading...." )
    print('\n\n')
    kartData = sh.worksheet('Total Scores').get_all_values()
    RaceCount = sh.worksheet('Race Count').get_all_values()
    KartScore = sh.worksheet('Kart Score').get_all_values()
    dfScores = pd.DataFrame(kartData[1:], columns=kartData[0])
    dfRaceCount = pd.DataFrame(RaceCount[1:], columns = RaceCount[0])
    dfKartScore = pd.DataFrame(KartScore[1:], columns = KartScore[0])
    inputRace(dfScores,dfRaceCount, dfKartScore, Track,Racers,Scores,TrackIndex)
 
 #edit a score
  elif selection == '2':
    print('Edit A Score is used to fix an error that may have occured while entering a race \nUse a positive number to increase a score, and a negative to decrease \nUse the net change you want to make as the value')
    Track = input("Name of Track: ")
    if Track in NickNameIndex:
      Track = NickNameIndex[Track]
    Racer = input("Names of Racer: ")
    Score = input("Score: ")
    print("\nYour selection of", selection, "is loading...." )
    print('\n\n')
    kartData = sh.worksheet('Total Scores').get_all_values()
    dfScores = pd.DataFrame(kartData[1:], columns=kartData[0])
    editAScore(dfScores, Track, Racer, Score,TrackIndex)

  #enter a winner
  elif selection == '3':
    Player = input("Winning Player: ")
    print("\nYour selection of", selection, "is loading...." )
    print('\n')
    Wins = sh.worksheet('GP Wins').get_all_values()
    dfWins = pd.DataFrame(Wins[1:], columns = Wins[0])
    enterWinner(dfWins, Player)
   
  #enter a dodge
  elif selection == '4':
    Player = input("Player Name: ")
    Count = input("How Many Dodges: ")
    print("\nYour selection of", selection, "is loading...." )
    print('\n')
    Shock = sh.worksheet('Shock Dodges').get_all_values()
    dfShock = pd.DataFrame(Shock[1:], columns = Shock[0])
    enterDodges(dfShock,Player,Count)

   #leaderboards
  elif selection == '5':
    print("\nYour selection of", selection, "is loading...." )
    kartData = sh.worksheet('Total Scores').get_all_values()
    RaceCount = sh.worksheet('Race Count').get_all_values()
    Wins = sh.worksheet('GP Wins').get_all_values()
    Shock = sh.worksheet('Shock Dodges').get_all_values()
    KartScore = sh.worksheet('Kart Score').get_all_values()
    dfKartScore = pd.DataFrame(KartScore[1:], columns = KartScore[0])
    dfScores = pd.DataFrame(kartData[1:], columns=kartData[0])
    dfRaceCount = pd.DataFrame(RaceCount[1:], columns = RaceCount[0])
    dfWins = pd.DataFrame(Wins[1:], columns = Wins[0])
    dfShock = pd.DataFrame(Shock[1:], columns = Shock[0])
    print('\n\n')
   
    print('Seeding Leaderboard')
    getSeedings(dfScores,dfRaceCount,dfWins,dfShock,dfKartScore,TrackIndex)
    print('\nPoints Per Race Leaderboard')
    getPointsPerRace(dfScores,dfRaceCount,TrackIndex)
    print('\nTotal Races Played')
    getRaceCountLeaderbaords(dfRaceCount, TrackIndex)
    print('\nGP Wins')
    getGPWinsLeaderboard(dfWins)
    print('\nShock Dodges')
    getShockDodges(dfShock)
 
  #print all data
  elif selection == '6':
    print("\nYour selection of", selection, "is loading...." )
    kartData = sh.worksheet('Total Scores').get_all_values()
    RaceCount = sh.worksheet('Race Count').get_all_values()
    Wins = sh.worksheet('GP Wins').get_all_values()
    Shock = sh.worksheet('Shock Dodges').get_all_values()
    dfScores = pd.DataFrame(kartData[1:], columns=kartData[0])
    dfRaceCount = pd.DataFrame(RaceCount[1:], columns = RaceCount[0])
    dfWins = pd.DataFrame(Wins[1:], columns = Wins[0])
    dfShock = pd.DataFrame(Shock[1:], columns = Shock[0])
    print('\nDatabase of total scores')
    print(dfScores)
    print('\nDatabase of Race Count')
    print(dfRaceCount)
    print('\nDatabase GP Wins')
    print(dfWins)
    print('\nDatabase of Shock Dodges')
    print(dfShock)
 
  #track data
  elif selection == '7':
    Track = input("Name of Track: ")
    if Track in NickNameIndex:
      Track = NickNameIndex[Track]
    print("\nYour selection of", selection, "is loading...." )
    print("\n\n")
    kartData = sh.worksheet('Total Scores').get_all_values()
    RaceCount = sh.worksheet('Race Count').get_all_values()
    dfScores = pd.DataFrame(kartData[1:], columns=kartData[0])
    dfRaceCount = pd.DataFrame(RaceCount[1:], columns = RaceCount[0])
    GetTrackData(dfScores,dfRaceCount,Track,TrackIndex)
  
  #racer data
  elif selection == '8':
    Racer = input("Name of Racer: ")
    print("\nYour selection of", selection, "is loading...." )
    print("\n\n")
    kartData = sh.worksheet('Total Scores').get_all_values()
    RaceCount = sh.worksheet('Race Count').get_all_values()
    Wins = sh.worksheet('GP Wins').get_all_values()
    Shock = sh.worksheet('Shock Dodges').get_all_values()
    KartScore = sh.worksheet('Kart Score').get_all_values()
    dfKartScore = pd.DataFrame(KartScore[1:], columns = KartScore[0])
    dfScores = pd.DataFrame(kartData[1:], columns=kartData[0])
    dfRaceCount = pd.DataFrame(RaceCount[1:], columns = RaceCount[0])
    dfWins = pd.DataFrame(Wins[1:], columns = Wins[0])
    dfShock = pd.DataFrame(Shock[1:], columns = Shock[0])
    getPlayerStats(dfScores, dfRaceCount, dfWins, dfShock, dfKartScore, Racer, TrackIndex)
 
  #All Track MVPs
  elif selection == '9':
    print("\nYour selection of", selection, "is loading...." )
    print('\n')
    kartData = sh.worksheet('Total Scores').get_all_values()
    RaceCount = sh.worksheet('Race Count').get_all_values()
    dfScores = pd.DataFrame(kartData[1:], columns=kartData[0])
    dfRaceCount = pd.DataFrame(RaceCount[1:], columns = RaceCount[0])
    print("Displaying all track MVPs...\n")

    getAllTrackOwners(dfScores,dfRaceCount,TrackIndex)

   
 #invalid selection
  else: 
    print('invalid selection restart the program')



  #####


  ##COMMENT THIS OUT WHEN TESTING

  #upadates the google sheet, a psyudo save of the data
  TotalScores = sh.worksheet('Total Scores')
  RaceCount = sh.worksheet('Race Count')
  WinCount = sh.worksheet('GP Wins')
  Shock = sh.worksheet('Shock Dodges')
  KartScore = sh.worksheet('Kart Score')
  
  #saves to the google sheet if nessassary, depending on the input of the user
  #saves if enter a race
  if selection =='1':
    set_with_dataframe(TotalScores, dfScores)
    set_with_dataframe(RaceCount,dfRaceCount)
    set_with_dataframe(KartScore,dfKartScore)
    print("Success - The Database Has Been Updated")
  #saves if edit a score
  elif selection =='2':
    set_with_dataframe(TotalScores, dfScores)
    print("Success - The Database Has Been Updated")
 #saves when winner added
  elif selection =='3':
    set_with_dataframe(WinCount,dfWins)
    print("Success - The Database Has Been Updated")
  #saves when shock dodges are counted
  elif selection =='4':
    set_with_dataframe(Shock,dfShock)
    print("Success - The Database Has Been Updated")


In [35]:
#This is now the cell you need to run the program
#The old main cell was redefined from the main cell to the singular method below
#This is to reduce the amount of scrolling needed when inputing or viewing the data

######IMPORTS######
import pandas as pd
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe
###################

#methodCall
RunKartniteStats()

Welcome to Kartnite Stats v2.2 
Developed by: Patrick Marinich

Below are the options, type the number of the option you would like
1. Input a Race 
2. Edit A Singular Score 
3. Enter A Grand Prix Winner 
4. Enter Shock Dodges 
5. View Leaderboards 
6. View All Databases 
7. View Track Records 
8. View Player Stats  
9. View All Current Track MVPs
What Would You Like To Do: 9

Your selection of 9 is loading....


Displaying all track MVPs...

                      Track          Current MVP
0             Luigi Circuit                  Pat
1           Moo Moo Meadows                  Pat
2            Mushroom Gorge                  Pat
3            Toad's Factory                  Pat
4             Mario Circuit                  Pat
5              Coconut Mall                  Pat
6                 DK Summit                  Pat
7         Wario's Gold Mine                  Pat
8             Daisy Circuit                Chris
9                Koopa Cape                  Pat
10            